In [27]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import metrics

import string
import spacy
np.random.seed(42)

In [28]:
import pandas as pd

# Load only the first 2000 rows of the CSV file
data = pd.read_csv(r"C:\Users\Vipul\train.csv", nrows=2000)


In [29]:
data.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [30]:
data.shape

(2000, 8)

In [31]:
data.drop(['id','severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace =True)

In [32]:
data.head(5)

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [33]:
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
print(stop_words)

{'who', 'latter', 'since', 'put', 'often', 'much', 'call', 'hers', 'others', 'are', 'am', 'move', 'them', 'show', 'everywhere', 'two', 'may', 'alone', 'all', "'m", 'indeed', 'herself', '‘s', 'almost', 'quite', 'some', 'please', 'become', 'next', 'due', 'she', 'see', 'thereafter', 'back', 'himself', 'about', 'might', 'third', 'his', 'whereas', 'used', 'neither', 'under', 'at', 'while', 'mine', 'thereupon', 'amount', 'he', 'within', '’ll', 'empty', '‘m', 'yet', 'nevertheless', '‘ve', 'have', 'whatever', 'hence', 'many', 'should', 'beyond', 'eight', 'was', 'above', 'the', 'twelve', 'doing', 'were', 'you', 'namely', 'only', 'then', 'hereby', 'or', 'itself', 'of', 'again', 'where', '’m', 'nowhere', 'also', 'unless', 'not', '‘re', 'whereby', 'hundred', 'our', 'really', 'your', 'very', 'nobody', "'re", 'through', 'onto', 'somehow', 'off', 'besides', 'several', 'more', 'just', 'thus', 'otherwise', 'that', 'without', 'to', 'whither', 'sometimes', 'yours', 'afterwards', 'therein', 'part', 'whole

In [34]:
punctuations = string.punctuation
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [35]:
def spacy_tokenizer(sentence):
    
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)



    # print(doc)
    # print(type(doc))

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # print(mytokens)

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens
     

In [36]:
sentence = "I am eating apple ?"
spacy_tokenizer(sentence)

['eat', 'apple']

In [37]:
count_vector = CountVectorizer(tokenizer = spacy_tokenizer)
# tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [38]:
count_vector.fit_transform(["I am eating apple, I like apple","I am playing cricket"]).toarray() 

C:\Users\Vipul\New_Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array([[2, 0, 1, 1, 0],
       [0, 1, 0, 0, 1]], dtype=int64)

In [39]:
count_vector.get_feature_names_out()

array(['apple', 'cricket', 'eat', 'like', 'play'], dtype=object)

In [40]:
count_vector.vocabulary_

{'eat': 2, 'apple': 0, 'like': 3, 'play': 4, 'cricket': 1}

In [41]:
from sklearn.model_selection import train_test_split

X = data['comment_text'] # the features we want to analyze
ylabels = data['toxic'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2,stratify=ylabels)
     

In [42]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

In [43]:
X_train_vetcors= count_vector.fit_transform(X_train)
X_test_vetcors= count_vector.transform(X_test)

C:\Users\Vipul\New_Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [44]:
X_train_vetcors.shape

(1600, 10921)

In [45]:
X_test_vetcors.shape

(400, 10921)

In [46]:
classifier.fit(X_train_vetcors,y_train)

LogisticRegression()

In [47]:
predicted = classifier.predict(X_test_vetcors)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.915
Logistic Regression Precision: 0.8
Logistic Regression Recall: 0.2


In [48]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)
X_train_vetcors= tfidf_vector.fit_transform(X_train)
X_test_vetcors= tfidf_vector.transform(X_test)

C:\Users\Vipul\New_Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [49]:
classifier = LogisticRegression()
classifier.fit(X_train_vetcors,y_train)
predicted = classifier.predict(X_test_vetcors)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))
     

Logistic Regression Accuracy: 0.915
Logistic Regression Precision: 1.0
Logistic Regression Recall: 0.15
